In [1]:
# Imports

import math
import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rnd
import warnings,datetime,os,calendar,csv,time

In [ ]:
import tensorflow as tf

In [ ]:
import pandas as pd
import seaborn as sns

from keras.models import Model,Sequential
from keras.layers import Dense,LSTM,Conv2D,Dropout,BatchNormalization,Input,Concatenate,Add,Activation,MaxPooling2D,AveragePooling2D,Flatten
import keras.backend as K

from sklearn import preprocessing as pp

from sklearn.cluster import KMeans,MeanShift
from sklearn.dummy import DummyClassifier,DummyRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier,AdaBoostRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.linear_model import BayesianRidge,Lasso,LinearRegression,SGDClassifier,SGDRegressor
from sklearn.mixture import BayesianGaussianMixture,GaussianMixture
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,RadiusNeighborsClassifier,RadiusNeighborsRegressor,NearestNeighbors
from sklearn.manifold import Isomap,TSNE
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.svm import LinearSVC,LinearSVR
from sklearn.neural_network import BernoulliRBM,MLPClassifier,MLPRegressor
from sklearn.decomposition import FactorAnalysis,KernelPCA,PCA,MiniBatchSparsePCA,FastICA
from sklearn.preprocessing import CategoricalEncoder,KBinsDiscretizer,LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler

from xgboost import XGBClassifier,XGBRegressor

import pickle,h5py,json

import pandas_datareader as pdr
import urllib
from bs4 import BeautifulSoup
import iexfinance as iex
from iexfinance.stocks import get_historical_data
from scipy.signal import resample,correlate
from scipy import fftpack

warnings.filterwarnings('ignore')
sns.set()
plt.xkcd()

In [2]:
start_date = datetime.datetime(2015,1,1)
end_date = datetime.datetime(2017,1,1)

with open('Documents\stock_symb.txt','r') as f:
    lines = f.readlines()
content = [line.strip() for line in lines]
tickers = [content[i] for i in list(np.random.permutation(np.arange(len(content))))]

In [3]:
ticker_set = tickers[:50]
print_progress = 10

DATA = []

ctr = 0
for ticker in ticker_set:

    if ctr%print_progress==0: print('Processing %d/%d'%(ctr,len(ticker_set)))
    ctr += 1
    try:
        df = get_historical_data(ticker,start=start_date,end=end_date,output_format='pandas')
    except:
        continue
    data = df['open'].values
    DATA.append(data)

Processing 0/50
Processing 10/50
Processing 20/50
Processing 30/50
Processing 40/50


In [4]:
# First project: Fill in NaNs

In [5]:
datalen = [len(data) for data in DATA]
most_common_length = max(set(datalen),key=datalen.count)

In [6]:
xtrain = [data for data in DATA if np.sum(np.isnan(data))==0 and len(data)==most_common_length]
xuse = [data for data in DATA if np.sum(np.isnan(data))!=0]

In [53]:
lookback = 5

S = []
for x in xtrain:
    for k in range(len(x)-lookback):
        S.append(x[k:k+lookback])

In [63]:
perc_nans = 0.5

X = []
Y = []

for s in S:
    num_nans = round(perc_nans*len(s))
    nan_indices = rnd.permutation(np.arange(0,len(s)))[:num_nans]
    ss = s.copy()
    ss[nan_indices]=-1
    X.append(ss)
    Y.append(s)
X = np.stack(X,axis=0)
Y = np.stack(Y,axis=0)

In [64]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y,test_size=0.2,shuffle=True)

In [77]:
in0 = Input(shape=(Xtrain.shape[1],))
x = Dense(units=10,activation='relu')(in0)
x = Dense(units=5,activation='relu')(x)
x = Dense(units=10,activation='relu')(x)
x = Dropout(rate=0.25)(x)
out0 = Dense(units=Xtrain.shape[1],activation='relu')(in0)

model = Model(in0,out0)
model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])

In [78]:
model.fit(Xtrain,Ytrain,batch_size=3,epochs=25,validation_split=0.1)

Train on 10059 samples, validate on 1118 samples
Epoch 1/25
10059/10059 [==============================] - 6s 560us/step - loss: 836.2879 - acc: 0.2222 - val_loss: 306.8449 - val_acc: 0.2093
Epoch 2/25
10059/10059 [==============================] - 6s 572us/step - loss: 356.1585 - acc: 0.1979 - val_loss: 303.4857 - val_acc: 0.2182
Epoch 3/25
10059/10059 [==============================] - 5s 530us/step - loss: 41.7583 - acc: 0.2096 - val_loss: 1.0673 - val_acc: 0.1503
Epoch 4/25
10059/10059 [==============================] - 5s 518us/step - loss: 1.0024 - acc: 0.1944 - val_loss: 1.0643 - val_acc: 0.2039
Epoch 5/25
10059/10059 [==============================] - 6s 601us/step - loss: 1.0178 - acc: 0.2081 - val_loss: 1.1134 - val_acc: 0.2576
Epoch 6/25
10059/10059 [==============================] - 6s 550us/step - loss: 1.0080 - acc: 0.2138 - val_loss: 1.1091 - val_acc: 0.1619
Epoch 7/25
10059/10059 [==============================] - 6s 560us/step - loss: 1.0195 - acc: 0.2185 - val_loss: 1

In [79]:
model.evaluate(Xtest,Ytest)

2795/2795 [==============================] - 0s 30us/step


[1.019156059788037, 0.2207513417028997]

In [80]:
Xpred = model.predict(Xtest)

In [81]:
Xtest[0]

array([74.8895, 75.0725, -1.    , -1.    , 74.6882])

In [82]:
Xpred[0]

array([74.09739 , 74.75027 , 74.541336, 75.560394, 75.37741 ],
      dtype=float32)

In [83]:
Ytest[0]

array([74.8895, 75.0725, 75.0908, 74.2031, 74.6882])